In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import face_recognition

# Define the folder where images are stored
FOLDER_NAME = 'images'
images, names = [], []
finger_tips = [8, 12, 16, 20] # Other than the thumb tip ('4')
thumb_tip = 4

# Load images and names from the folder
myList = os.listdir(FOLDER_NAME) # This will take all the items in that folder and store them into a list
print("Images Found:", myList)
for numImg in myList:
    curImg = cv2.imread(f'{FOLDER_NAME}/{numImg}') # Read each image
    images.append(curImg) # Add images to the list
    names.append(os.path.splitext(numImg)[0]) # Store names without file extensions

# Initialize Mediapipe hands model
mp_holistic = mp.solutions.holistic
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# User preferences for optional features
runit = input("Do you want to include hand landmarks? (yes/no): ") == "yes"
runit2 = input("Do you want contours? (yes/no): ") == "yes"
runit3 = input("Do you want sounds? (yes/no): ") == "yes"

# Function to encode face images
def encoding(imagesc):
    encodeList = []
    for img in imagesc:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0] # Get face encodings
        encodeList.append(encode)
    return encodeList

knownEncodeList = encoding(images) # Generate known encodings
print('Face encoding completed')

# Function to detect hand landmarks
def hand_landmark_finder(img, h, w, c):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB) # Process the hand landmarks
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lm_list = [lm for lm in handLms.landmark]
            finger_fold_status = []
            for tip in finger_tips:
                x, y = int(lm_list[tip].x * w), int(lm_list[tip].y * h)
                cv2.circle(img, (x, y), 15, (0, 255, 0), cv2.FILLED) # Draw circles on fingertips
                if lm_list[tip].x < lm_list[tip - 3].x:
                    cv2.circle(img, (x, y), 15, (255, 0, 0), cv2.FILLED)
                    finger_fold_status.append(True)
                else:
                    finger_fold_status.append(False)
            if lm_list[thumb_tip].y < lm_list[thumb_tip - 1].y < lm_list[thumb_tip - 2].y:
                print("Like")
                cv2.putText(img, "Like", (30, 20), cv2.FONT_ITALIC, 1, (0, 255, 0), 3)
                if runit3:
                    os.system("say 'yay'")
            elif lm_list[thumb_tip].y > lm_list[thumb_tip - 1].y > lm_list[thumb_tip - 2].y:
                print("Dislike")
                cv2.putText(img, "Dislike", (30, 20), cv2.FONT_ITALIC, 1, (255, 0, 0), 3)
                if runit3:
                    os.system("say 'aww man'")
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS) # Draw hand connections

# Start video capture
cap = cv2.VideoCapture(0) # Start recording

while True:
    success, img = cap.read()
    h, w, c = img.shape
    imgShape = cv2.resize(img, (0, 0), None, 0.25, 0.25) # Resize the video for faster processing
    imgShape = cv2.cvtColor(imgShape, cv2.COLOR_BGR2RGB)
    frameFace = face_recognition.face_locations(imgShape) # Find face locations
    encodeFrame = face_recognition.face_encodings(imgShape, frameFace) # Get face encodings

    # Draw contours if enabled
    if runit2:
        imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 100, 255, 0) # Apply thresholding
        contours, hierarchy = cv2.findContours(thresh, 1, 2)
        cv2.drawContours(img, contours, -1, (0, 0, 0), 3) # Draw contours

    # Recognize faces
    for ef, fl in zip(encodeFrame, frameFace):
        matches = face_recognition.compare_faces(knownEncodeList, ef)
        faceDis = face_recognition.face_distance(knownEncodeList, ef)
        matchIndex = np.argmin(faceDis) # Get index of the closest match
        if matches[matchIndex]:
            name = names[matchIndex].upper()
            print(name)
            y1, x2, y2, x1 = [i * 4 for i in fl] # Scale back up
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2) # Draw rectangle around face
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 0), cv2.FILLED) # Add a filled rectangle below face
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_ITALIC, 1, (255, 255, 255), 2) # Add name text
            if runit3:
                os.system(f"say '{name.lower()}'") # Speak the name

    # Detect hand landmarks if enabled
    if runit:
        hand_landmark_finder(img, h, w, c)

    # Display the output
    cv2.imshow('Camera', img)
    cv2.waitKey(1) # Keep running until user exits

# AMAZING information source: https://docs.opencv.org/4.x/index.html
